In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/Shareddrives/CSE573/preprocessed
%ls

/content/drive/Shareddrives/CSE573/preprocessed
AaplAmzn_NewsDateDirection.csv
Aapl_NewsPriceHistDirection.csv
Aapl_SentimentAggMeanPriceHistDirection.csv
Aapl_SentimentAggSumPriceHistDirection.csv
aapl_text_direction_date.csv
Amzn_NewsPriceHistDirection.csv
Amzn_SentimentAggMeanPriceHistDirection.csv
Amzn_SentimentAggSumPriceHistDirection.csv
amzn_title_processedText_direction.csv
amzn_title_processedText_direction_date.csv
amzn_title_text_label.csv
lstm_amzn_fulltext/


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

import requests
import io

In [ ]:
df_aapl = pd.read_csv('aapl_text_direction_date.csv')


In [ ]:
df_aapl

,title,text,direction,prediction_date,text_without_stopwords,processedText
0,The evidence is mounting that Apple&apos;s nex...,"september 2, 2018 admin trading ideas comments...",1,2018-09-04,september 2 2018 admin trading ideas comments ...,september 2 2018 trading ideas comments eviden...
1,MARKET SNAPSHOT: Stocks Sag With Nafta In Focu...,"by mark decambre, marketwatch , ryan vlastelic...",1,2018-09-04,mark decambre marketwatch ryan vlastelica s p ...,mark decambre marketwatch ryan vlastelica p 50...
2,Tracking David Einhorn's Portfolio - Q2 2018 U...,this article is part of a series that provides...,1,2018-09-04,article part series provides ongoing analysis ...,article part series provides ongoing analysis ...
3,Multiplying my 247% Profit in Pot Stocks,"frankly, i’ve just developed a brand-new tradi...",1,2018-09-04,frankly i ’ ve developed brand-new trading str...,i ve developed trading allowing us profit amer...
4,Mike Khouw's Apple Options Trade,"on cnbc's ""options action"" , mike khouw recomm...",1,2018-09-04,cnbc 's `` options action '' mike khouw recomm...,cnbc `` options action `` mike recommended lon...
...,...,...,...,...,...,...
9995,Advantages of the Individual Investor - Wealth...,advantages of the individual investor posted o...,-1,2018-06-07,advantages individual investor posted wealthy ...,advantages individual investor posted content ...
9996,Global Seed Processing Machine Market 2018 Per...,search for: global seed processing machine mar...,-1,2018-06-07,search for global seed processing machine mark...,search for global processing machine market 20...
9997,Dark mode looks great on the Mac — but it's iO...,advertisement business dark mode looks great o...,-1,2018-06-07,advertisement business dark mode looks great m...,advertisement business mode looks great mac — ...
9998,UBS Group Analysts Give Apple (AAPL) a $210.00...,tweet\napple (nasdaq:aapl) received a $210.00 ...,-1,2018-06-07,tweet apple nasdaq aapl received 210.00 price ...,tweet apple nasdaq aapl received 210.00 price ...


In [ ]:
df_aapl = df_aapl.dropna()

In [ ]:
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer


In [ ]:

text_aapl=df_aapl['processedText'].tolist()
label_aapl=df_aapl['direction'].to_numpy()



print(text_aapl[0])
print(label_aapl[0])

september 2 2018 trading ideas comments evidence apple next big product pair computer glasses aapl apple said earlier week bought small colorado company worked augmented reality glasses it pretty clear point apple working pair course apple working new products 5 billion headquarters research labs
1


In [ ]:
#class_label_amzn = (label_amzn + 1) / 2
class_label_aapl = pd.get_dummies(label_aapl).values

In [ ]:

text_tokenized_aapl, text_tokenizer_aapl=tokenize(text_aapl)

text_padded_aapl=pad_sequences(text_tokenized_aapl, padding='post')

In [ ]:
#print(title_tokenized_amzn[0])
print(text_padded_aapl[0])

[176  15  20 ...   0   0   0]


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(text_padded_aapl, class_label_aapl, test_size=0.2, random_state=56)

In [ ]:
TrainX = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
TestX = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
TrainX = TrainX.astype('float64')
TestX = TestX.astype('float64')

In [ ]:
TrainX[0:5]

array([[[4.000e+00, 1.000e+01, 1.990e+02, ..., 0.000e+00, 0.000e+00,
         0.000e+00]],

       [[1.150e+02, 2.270e+02, 3.500e+02, ..., 0.000e+00, 0.000e+00,
         0.000e+00]],

       [[3.154e+03, 3.670e+03, 5.340e+02, ..., 0.000e+00, 0.000e+00,
         0.000e+00]],

       [[1.083e+03, 1.000e+01, 1.560e+02, ..., 0.000e+00, 0.000e+00,
         0.000e+00]],

       [[1.510e+02, 3.450e+02, 3.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00]]])

In [ ]:
TrainX.shape

(7997, 1, 10268)

In [ ]:
Y_train.shape

(7997, 2)

In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import Recall, Precision

In [ ]:
layers = [LSTM(128,input_shape=(1,TrainX.shape[2]),dropout=0.2), Dense(2,activation='sigmoid')]
model = tf.keras.models.Sequential(layers)
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(),metrics=['accuracy', Recall(), Precision()])
model.fit(TrainX, Y_train, epochs=40, batch_size=64, verbose=2,validation_split=0.2)

Epoch 1/40
100/100 - 6s - loss: 0.7281 - accuracy: 0.5093 - recall: 0.5301 - precision: 0.5033 - val_loss: 0.7019 - val_accuracy: 0.4994 - val_recall: 0.5869 - val_precision: 0.5054 - 6s/epoch - 64ms/step
Epoch 2/40
100/100 - 4s - loss: 0.6995 - accuracy: 0.5210 - recall: 0.3871 - precision: 0.5076 - val_loss: 0.7031 - val_accuracy: 0.4913 - val_recall: 0.1937 - val_precision: 0.4851 - 4s/epoch - 41ms/step
Epoch 3/40
100/100 - 4s - loss: 0.6997 - accuracy: 0.5154 - recall: 0.4705 - precision: 0.5033 - val_loss: 0.6933 - val_accuracy: 0.5213 - val_recall: 0.5350 - val_precision: 0.4968 - 4s/epoch - 41ms/step
Epoch 4/40
100/100 - 4s - loss: 0.6961 - accuracy: 0.5206 - recall: 0.7200 - precision: 0.5037 - val_loss: 0.6999 - val_accuracy: 0.5019 - val_recall: 0.8031 - val_precision: 0.5014 - 4s/epoch - 37ms/step
Epoch 5/40
100/100 - 4s - loss: 0.6908 - accuracy: 0.5310 - recall: 0.7388 - precision: 0.5099 - val_loss: 0.6982 - val_accuracy: 0.5181 - val_recall: 0.6244 - val_precision: 0.503

In [ ]:
eva = model.evaluate(TestX, Y_test, verbose=2)

63/63 - 1s - loss: 0.6966 - accuracy: 0.4895 - recall: 0.9930 - precision: 0.4999 - 1s/epoch - 21ms/step


In [ ]:
f1 = 2 * ((eva[2]*eva[3])/(eva[2]+eva[3]))

In [ ]:
f1

0.664992454510558

In [ ]:
model.save("../models/lstm_aapl_fulltext")